In [308]:
# disable global logging from the virtual coach
import logging
logging.disable(logging.INFO)
logging.getLogger('rospy').propagate = False
logging.getLogger('rosout').propagate = False

In [309]:
# log into the virtual coach, update with your credentials
try:
    from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
    vc = VirtualCoach(environment='local')
except ImportError as e:
    print(e)
    print("You have to start this notebook with the command:\
          cle-virtual-coach jupyter notebook")
    raise e

In [347]:
record_success_csv = """
import numpy as np

@nrp.MapSpikeSink("actors", nrp.map_neurons(range(0, 3), lambda i: nrp.brain.actors[i]), nrp.population_rate)
@nrp.Robot2Neuron()
def record_index_csv(t, actors):

    clientLogger.info(actors.rate)
    clientLogger.info(np.argmax(actors.rate))

"""

In [393]:
img_to_mugs = """

import numpy as np
import cv2
import sensor_msgs

@nrp.MapRobotSubscriber("img_msg", Topic("/icub_model/left_eye_camera/image_raw", sensor_msgs.msg.Image))
@nrp.MapVariable("area_grid", initial_value=None, scope=nrp.GLOBAL)
@nrp.MapVariable("estimate", initial_value=None, scope=nrp.GLOBAL)
@nrp.MapSpikeSource("grid_neurons", nrp.map_neurons(range(0, 9), lambda i: nrp.brain.sensors[i]), nrp.poisson)
@nrp.Robot2Neuron()
def img_to_mug_center(t, img_msg, area_grid, estimate, grid_neurons):
    
    if img_msg.value is None:
        return
        
    img = CvBridge().imgmsg_to_cv2(img_msg.value, "bgr8")
    most_vibrant_channel = np.argmax(img, axis=2)
    
    # Filter red 
    img[most_vibrant_channel != 2] = 0
    img[img[:,:,2] < 150] = 0
    red = img[:, :, 2]
    _, thresh = cv2.threshold(red, 150, 255, 0)
    kernel = np.ones((5,5), np.uint8)
    thresh = cv2.erode(thresh, kernel,iterations = 1)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contour_thresh = 50
    contours = [c for c in contours if cv2.contourArea(c) > contour_thresh]

    moments = [cv2.moments(c) for c in contours]
    centers = sorted([(int(M['m10']/M['m00']), int(M['m01']/M['m00'])) for M in moments])
    center_points = np.array(centers)
    
    if center_points is None or center_points.shape != (3, 2):
        return
        
    if area_grid.value is None:

        x_intervals = [int(center_points[0, 0] + 0.5 * \
                        (center_points[1, 0] - center_points[0, 0])),
                       int(center_points[1, 0] + 0.5 * \
                        (center_points[2, 0] - center_points[1, 0]))]
        y_intervals = []
        for cnt in contours:
            (_,y),radius = cv2.minEnclosingCircle(cnt)
            y_intervals.extend([int(y + 1.5 * radius), int(y - 1.5 * radius)])
        y_intervals = [min(y_intervals), max(y_intervals)]
        
        area_grid.value = x_intervals, y_intervals
        
    if estimate.value is None or estimate.value == [False] * 3:
        estimate.value = [False] * 3
        correct_id = np.argmin(np.transpose(center_points), axis=1)[1]
        clientLogger.info(correct_id)
        estimate.value[correct_id] = True  
        
    def map_border(idx, x_or_y):
        if idx == 0:
            return slice(0, area_grid.value[x_or_y][0])
        elif idx == 1:
            return slice(area_grid.value[x_or_y][0], area_grid.value[0][1])
        elif idx == 2:
            return slice(area_grid.value[x_or_y][1], -1)
         
    locations = np.zeros((3,3))
    if area_grid.value is not None:
        for center in center_points:
            x_idx = np.searchsorted(area_grid.value[0], center[0])
            y_idx = np.searchsorted(area_grid.value[1], center[1])
            img_part = red[map_border(y_idx, x_or_y=1), map_border(x_idx, x_or_y=0)]
            locations[y_idx, x_idx] = np.sum(np.ones_like(img_part)) / np.sum(np.ones_like(red)) * 100.
            #if True in estimate.value and estimate.value.index(True) == x_idx: 
            #    locations[1, x_idx] = 500.
                
    grid_neurons.rate = locations.flatten()
    clientLogger.info(locations)
    
    
    return CvBridge().cv2_to_imgmsg(img, 'bgr8')

"""

In [387]:
import cv2
import numpy as np
import rospy
import time
from cv_bridge import CvBridge
import sensor_msgs
import std_msgs
from gazebo_msgs.srv import GetModelState, SetModelState
from gazebo_msgs.msg import ModelState
from thread import start_new_thread
import math
import thimblerigger_config as tc
from std_srvs.srv import Trigger, TriggerResponse
from std_msgs.msg import UInt32MultiArray
from std_msgs.msg import MultiArrayDimension
import tempfile
import os
import csv

%matplotlib inline

class RobotMover(object):

    def __init__(self, model_name="robot"):
        self.model_name = model_name

    def go_to_pose(self, x, y, orientation):
        pass


class TeleportRobotMover(RobotMover):

    def __init__(self, model_name="robot"):
        self.get_position = rospy.ServiceProxy("/gazebo/get_model_state", GetModelState)
        self.set_position = rospy.ServiceProxy("/gazebo/set_model_state", SetModelState)
        super(TeleportRobotMover, self).__init__(model_name=model_name)

    def go_to_pose(self, x, y, orientation):
        current_robot_pose = self.get_position(self.model_name, "")
        new_state = ModelState()
        new_state.model_name = self.model_name
        new_state.pose = current_robot_pose.pose
        new_state.scale = current_robot_pose.scale
        new_state.twist = current_robot_pose.twist
        new_state.pose.position.x = x
        new_state.pose.position.y = y
        new_state.pose.orientation.x = orientation[1]
        new_state.pose.orientation.y = orientation[2]
        new_state.pose.orientation.z = orientation[3]
        new_state.pose.orientation.w = orientation[0]
        new_state.reference_frame = "world"
        self.set_position(new_state)


class ChallengeInteractor(object):

    def __init__(self):
        rospy.wait_for_service(tc.thimblerigger_show_correct_service, 10)
        print("Show mug service available...")
        rospy.wait_for_service(tc.thimblerigger_hide_correct_service, 10)
        print("Hide mug service available...")
        rospy.wait_for_service(tc.thimblerigger_shuffle_service, 10)
        print("Shuffle service available...")
        print("All thimblerigger services found.")                       
        self.show_correct_mug = rospy.ServiceProxy(tc.thimblerigger_show_correct_service, Trigger)
        self.hide_correct_mug = rospy.ServiceProxy(tc.thimblerigger_hide_correct_service, Trigger)
        self.shuffle = rospy.ServiceProxy(tc.thimblerigger_shuffle_service, Trigger)

                               
class Solver(object):

    def __init__(self, sim, data_dir):
        
        
        self.neuron_grid = None
        self.define_neuron_grid = False

        self.sim = sim
        self.sim.start()
        self.data_dir = data_dir
        
        self.estimate = [False] * 3
        self.center_points = None
        self.robot_mover = TeleportRobotMover()
        self.current_view = "front"
        self.challenge = ChallengeInteractor()

        self.set_joints_start()
        print("Moved robot to start pose")

    def set_joints_start(self):
        neck = rospy.Publisher("/robot/neck_pitch/pos", std_msgs.msg.Float64, queue_size=1)
        l_elbow = rospy.Publisher("/robot/l_elbow/pos", std_msgs.msg.Float64, queue_size=1)
        r_elbow = rospy.Publisher("/robot/r_elbow/pos", std_msgs.msg.Float64, queue_size=1)
        def lower_gaze_and_arms():
            while not rospy.is_shutdown():
                neck.publish(std_msgs.msg.Float64(-0.8))
                l_elbow.publish(std_msgs.msg.Float64(-25.0))
                r_elbow.publish(std_msgs.msg.Float64(-25.0))
            l_elbow.unregister()
            r_elbow.unregister()
        start_new_thread(lower_gaze_and_arms, ())
        time.sleep(2)
        self.side_look()
        time.sleep(2)

        
    def __enter__(self):
        return self
    
    def __exit__(self, *args, **kwargs):
        print("Closing windows...")
        self.sim.pause()
        with open(os.path.join(self.data_dir, "index_real_estimate.csv"), 'wb') as f:
            cf = csv.writer(f)
            csv_data = self.sim.get_csv_data("index_real_estimate.csv")
            cf.writerows(csv_data)
            
        self.activations.unregister()
        self.sim.stop()

        print("destroying the solver...")


    def front_look(self):
        self.current_view = "front"
        self.robot_mover.go_to_pose(x=-0.75, y=0., orientation=(0, 0, 1, 0))

    def side_look(self):
        self.current_view = "side"
        orientation = (math.sqrt(0.5), 0, 0, -math.sqrt(0.5))
        self.robot_mover.go_to_pose(x=0.4, y=-0.9, orientation=orientation)


    def find_correct_mug_beginning(self):
        self.allow_define_neuron_grid()
        self.challenge.show_correct_mug()
        print("checking which mug contains the ball")
        #correct_mug_id = np.argmin(np.transpose(self.center_points), axis=1)[1]
        #self.estimate[correct_mug_id] = True
        #print("Found this mug layout: ", self.estimate)
        self.challenge.hide_correct_mug()
        print("found correct mug beginning.")
        
    def shuffle(self):
        print("Shuffling mugs now")
        self.challenge.shuffle()

    def verify_guess(self):
        self.side_look()
        self.challenge.show_correct_mug()
        correct_mug_id = np.argmin(np.transpose(self.center_points), axis=1)[1]
        self.challenge.hide_correct_mug()
        return self.estimate.index(True) == correct_mug_id

In [388]:
def run_experiment(data_dir):
    sim = vc.launch_experiment('ExDPerceptionChallengeKIT')
    solver = Solver(sim, data_dir)
    return solver


In [396]:
tmp_folder = tempfile.mkdtemp()
solver = run_experiment(data_dir=tmp_folder)

[{u'gzweb': {u'assets': u'http://localhost:8080/assets',
             u'nrp-services': u'http://localhost:8080',
             u'videoStreaming': u'http://localhost:8080/webstream/',
             u'websocket': u'ws://localhost:8080/gzbridge'},
  u'id': u'localhost',
  u'rosbridge': {u'websocket': u'ws://localhost:8080/rosbridge'},
  u'serverJobLocation': u'local'}]
Show mug service available...
Hide mug service available...
Shuffle service available...
All thimblerigger services found.
Moved robot to start pose


In [397]:
solver.sim.add_transfer_function(record_success_csv) # This breaks the neurons
solver.sim.add_transfer_function(img_to_mugs)
time.sleep(2)
solver.challenge.show_correct_mug()


success: True
message: show_mug_with_ball finished successfully.

In [282]:
solver.shuffle()

Shuffling mugs now


In [164]:
with solver:
    pass


Closing windows...
destroying the solver...


In [22]:
import pandas
csv_file = os.path.join(tmp_folder, "index_real_estimate.csv")
data_frame = pandas.read_csv(csv_file)
score = np.sum(data_frame["real"]==data_frame["estimate"])
print(data_frame)
print(score)
print(score / len(data_frame["real"]))

        Time  real  estimate
0      15.20     1         0
1      15.22     1         0
2      15.24     1         0
3      15.26     1         0
4      15.28     1         0
5      15.30     1         0
6      15.32     1         0
7      15.34     1         0
8      15.36     1         0
9      15.38     1         0
10     15.40     1         0
11     15.42     1         0
12     15.44     1         0
13     15.46     1         0
14     15.48     1         0
15     15.50     1         0
16     15.52     1         0
17     15.54     1         0
18     15.56     1         0
19     15.58     1         0
20     15.60     1         0
21     15.62     1         0
22     15.64     1         0
23     15.66     1         0
24     15.68     1         0
25     15.70     1         0
26     15.72     1         0
27     15.74     1         0
28     15.76     1         0
29     15.78     1         0
...      ...   ...       ...
8339  181.98     1         0
8340  182.00     1         0
8341  182.02  